In [1]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image
from skimage import transform

import os
import numpy as np
import glob

In [ ]:
parasitized = glob.glob('dataset/parasitized/*.png')
uninfected = glob.glob('dataset/uninfected/*.png')

print("Parasitized data: {}\nUninfected data: {}".format(len(parasitized), len(uninfected)))

Parasitized data: 1000
Uninfected data: 1000


### Preprocessing

#### Data Augmentation

In [ ]:
dimension = 128
batch = 32
data_dir = "dataset"

datagen = ImageDataGenerator(rescale=1/255.0, 
                             validation_split=0.2,
                             zoom_range=0.05,
                             width_shift_range=0.05,
                             height_shift_range=0.05,
                             shear_range=0.05,
                             horizontal_flip=True)

train_data = datagen.flow_from_directory(data_dir, 
                                         target_size=(dimension, dimension), 
                                         batch_size=batch, 
                                         class_mode='categorical', 
                                         subset='training')

validation_data = datagen.flow_from_directory(data_dir,
                                              target_size=(dimension, dimension),
                                              batch_size=batch,
                                              class_mode='categorical',
                                              subset='validation',
                                              shuffle=False)

test_data = datagen.flow_from_directory(data_dir,
                                        target_size=(dimension, dimension),
                                        batch_size=1,
                                        shuffle=False)

print("Indexes class labels: ", train_data.class_indices)
print("Number of training data: ", len(train_data))
print("Number of validation data: ", len(validation_data))
print("Number of testing data: ", len(test_data))

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Indexes class labels:  {'Parasitized': 0, 'Uninfected': 1}
Number of training data:  50
Number of validation data:  13
Number of testing data:  2000


#### Modeling Data Training

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(dimension, dimension, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

# Compile the model
model.compile(optimizer="adam", 
              loss="binary_crossentropy", 
              metrics=["accuracy"])

# Save the best trained model by monitoring validation loss
model_name = "train_data_model.b"
model_checkpoint = ModelCheckpoint(model_name, 
                                   save_weights_only=False, 
                                   monitor='val_loss', 
                                   verbose=1, 
                                   mode='auto', 
                                   save_best_only=True)

# Training dataset
history = model.fit(train_data, 
                    batch_size=batch, 
                    epochs=30, 
                    validation_data=validation_data,
                    callbacks = [model_checkpoint],
                    verbose=1)

Epoch 1/30
50/50 [==============================] - 385s 8s/step - loss: 0.6946 - accuracy: 0.5412 - val_loss: 0.6757 - val_accuracy: 0.5750

Epoch 00001: val_loss improved from inf to 0.67573, saving model to /content/drive/MyDrive/malaria/train_data_model.b
INFO:tensorflow:Assets written to: /content/drive/MyDrive/malaria/train_data_model.b/assets
Epoch 2/30
50/50 [==============================] - 37s 738ms/step - loss: 0.6539 - accuracy: 0.6275 - val_loss: 0.7431 - val_accuracy: 0.6050

Epoch 00002: val_loss did not improve from 0.67573
Epoch 3/30
50/50 [==============================] - 37s 730ms/step - loss: 0.6077 - accuracy: 0.6781 - val_loss: 0.5729 - val_accuracy: 0.8025

Epoch 00003: val_loss improved from 0.67573 to 0.57291, saving model to /content/drive/MyDrive/malaria/train_data_model.b
INFO:tensorflow:Assets written to: /content/drive/MyDrive/malaria/train_data_model.b/assets
Epoch 4/30
50/50 [==============================] - 37s 728ms/step - loss: 0.5003 - accuracy: 0

### Evaluate Model

In [ ]:
# Load compiled model
model = load_model("train_data_model.b")
model.evaluate(test_data, verbose=0)

[0.11002203077077866, 0.9639999866485596]

In [ ]:
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('int32')/255
   np_image = transform.resize(np_image, (128, 128, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [ ]:
def prediction(image):
  np_image = load(image)
  prediction = model.predict(np_image) # Predict input image
  result = [1 * (x[0]>=0.5) for x in prediction] # Normalize prediction
  if result[0] == 0:
    return False
  return True

In [ ]:
test1 = prediction("test-data/parasitized1.png")
test2 = prediction("test-data/parasitized2.png")
test3 = prediction("test-data/parasitized3.png")
test4 = prediction("test-data/parasitized4.png")
test5 = prediction("test-data/uninfected1.png")
test6 = prediction("test-data/uninfected2.png")
test7 = prediction("test-data/uninfected3.png")
test8 = prediction("test-data/uninfected4.png")

print("Result for infected or uninfected of malaria")
print("Test 1: ", test1)
print("Test 2: ", test2)
print("Test 3: ", test3)
print("Test 4: ", test4)
print("Test 5: ", test5)
print("Test 6: ", test6)
print("Test 7: ", test7)
print("Test 8: ", test8)

Result for infected or uninfected of malaria
Test 1:  True
Test 2:  True
Test 3:  True
Test 4:  True
Test 5:  False
Test 6:  False
Test 7:  False
Test 8:  False
